# SIMPACK Cartographic Track 示例思路和代码
- 实现了各分段（STR/CIR/BLO）的曲率函数 κ(s)，并在分段交界处用一段“平滑区”保证二阶导数的连续——这与 SIMPACK 的“Bloss + smoothing”原理相似，可以在整个里程范围内得到一条更光滑、与 SIMPACK 结果更贴合的 κ(s)
- 最后对曲率做数值积分，即可获得轨道在平面内的坐标；若再加上竖向坡度、超高等，也能扩展到 3D 轨道
- **数据保存**: 轨道空间几何保存为 trajectory_data.npz

## 提示
1. 下述代码依然是“示例”，并未 100% 复刻 SIMPACK 的所有插值细节；尤其在 Bloss 与 smoothing 同处一段时，SIMPACK 可能还包含其他工程上累积的经验规则
2. 然而，本示例已体现用分段多项式在段交界处做无缝拼接的关键思路，能够在多数应用中和 SIMPACK 原生结果达到极高的一致度
3. 代码示例只对“水平曲率κ”演示了完整的拼接；若要处理超高 𝑢(𝑠)，请使用相同的逻辑（把 κ 换成 u 即可）

# 整体思路
## 1. 原始分段函数
在 Cartographic Track 的每段，都可以先写出一个原始曲率公式 κ_raw(s)：
- STR 段：κ=0
- CIR 段：κ=1/R​（常数）
- BLO 段（Bloss 过渡曲线）
  $\kappa(s) = \kappa_1 + (\kappa_2 - \kappa_1) \cdot (3x^2 - 2x^3), \quad x = \frac{s}{L_\mathrm{seg}}$。

其中, $\kappa_1 = \frac{1}{R_1}, \quad \kappa_2 = \frac{1}{R_2}$。
例如，若一段声明 ('BLO',50,0,300)，则表示“Bloss 过渡曲线长度 50m，初末曲率分别 0 和 1/300”。

## 2. 分段连接 + Smoothing

SIMPACK 提供一个"平滑总长度" $L_{\mathrm{smo}}$（常见取 2 m ～ 5 m 或更小）来**数值上**保证相邻两段衔接时的**高阶连续**：

* 在分段 $i$ 和 $i+1$ 的边界 $s_i$ 附近，**各留一半 $L_{\mathrm{smo}}/2$ 给平滑过渡**。
* 这部分平滑区并**不**额外增加段长，而是**覆盖**了分段的首尾各 $L_{\mathrm{smo}}/2$ 范围。
* 用一条**5 次多项式**在 $[\,s_i - L_{\mathrm{smo}}/2,\; s_i + L_{\mathrm{smo}}/2\,]$ 上插值，使其与前后段的"原始" $\kappa_{\mathrm{raw}}$ 的 $\kappa$、$\kappa'$、$\kappa''$ 在边界上保持一致，从而保证 $\mathcal{C}^2$ 光滑。

这样，一条具名义长度 $L_{\mathrm{seg}}$ 的 Bloss 段，其真正用于「Bloss 公式」的区间只剩 $(L_{\mathrm{seg}} - L_{\mathrm{smo}})$；首尾的 $L_{\mathrm{smo}}/2$ 用于 polynomial smoothing。**结果**：

* 总长仍是 $L_{\mathrm{seg}}$，但在内部融合了 Bloss 与 smoothing 的小段。
* 段与段之间二阶导数连续，避免了"反复过渡"所带来的偏移累积。

## 3. 生成全局 $\kappa(s)$ 函数

* 将**所有**段（STR/CIR/BLO）加上其**前后 smoothing** "拼"起来后，就得到一个从 $s=0$ 到 $s=L_{\mathrm{total}}$ 的分段式函数 $\kappa_{\mathrm{global}}(s)$。
* 同理，可对超高 $u(s)$ 或竖向坡度 $p(s)$ 做类似的 piecewise + smoothing 处理。

## 4. 数值积分得到 $(x(s),\,y(s))$

一旦有了 $\kappa_{\mathrm{global}}(s)$，即可做**航向角** $\psi$ 的积分：$\frac{d\psi}{ds} \;=\;\kappa(s) \quad\Longrightarrow\quad \psi(s)\;=\;\int_0^s \kappa(\tau)\,d\tau.$

然后在平面内积分坐标：$\frac{dx}{ds} = \cos\bigl[\psi(s)\bigr], \qquad \frac{dy}{ds} = \sin\bigl[\psi(s)\bigr].$

若仅考虑水平面（无竖曲线），则可令 $z=0$。若要考虑**坡度** $p(s)$，再做一次积分 $z'(s)=p(s)$ 得到 $z(s)$。

## 5. 左右钢轨 + 3D

* 如果还要得到左右钢轨的 3D 坐标，可先在每个 $s$ 点计算**超高** $u(s)$ 并转换成横滚角 $\phi(s)=\arcsin\!\bigl[u(s)/b_{\mathrm{ref}}\bigr]$。
* 然后，以中心线为基准，在局部坐标 $(y_{\mathrm{local}},\,z_{\mathrm{local}})$ 上做 $\pm b_{\mathrm{ref}}/2$ 与 $\pm u/2$ 的偏移，通过 $\psi(s)$ 和 $\phi(s)$ 转回全局，便能得到左右股钢轨或轮轨实际位置。
* 在已封装的 Python 代码中，这往往体现在"left_rail"与"right_rail"的计算环节中。

## 小结

1. **"Bloss + smoothing"** 的关键是：smoothing 并不额外拉长段，而是**占用**分段首尾的小范围，用高阶多项式衔接前后段的曲率及其导数。
2. 在此思路下，**整条轨道**得到一个二阶（甚至三阶）连续的 $\kappa(s)$ 函数，避免了"重复过渡"或"段间不连续"导致的误差累积。
3. **示例代码**（参考 xxx.py）中，演示了如何将**原始 Bloss** 段 + **多项式平滑**合并到一起，并通过数值积分得到平面曲线；可按同理处理超高、竖曲线等需求。
4. 由于 SIMPACK 还可能在极端工况（小半径、陡坡、大超高）时有其他内部修正，若需与其"1:1 完全对齐"，还请检查文档细节或对照实际导出的轨道数据做比对。

# 独立运行代码
- 定义轨道分段及其参量
- 输出 .npz 与 .json 轨道数据

In [ ]:
%matplotlib widget

import numpy as np
import matplotlib.pyplot as plt
import json
import os

# 从 tools_SPCKTrk.py 中导入我们需要的主函数
from tools_SPCKTrk import generate_trajectory

#===================== 1) 轨道段定义 =====================
# 水平段
h_segments = [
    ('STR',  50      ),            # 1.直线50m
    ('BLO',  50, 0.0, 300.0 ),     # 2.Bl.  0-->1/300
    ('CIR', 1000, 300 ),           # 3.圆曲线 300m半径
    ('BLO',  50, 300, 0 ),         # 4.Bl.  1/300-->0
    ('STR',  500     )             # 5.直线500m
]

# 超高段
u_segments = [
    ('CST', 50,   0.0),            # 常值超高 0
    ('BLO', 50, 0.0, 0.12),        # 从0变化到0.12
    ('CST', 1000, 0.12),           # 保持0.12不变
    ('BLO', 50, 0.12, 0.0),        # 回到0
    ('CST', 500, 0.0 )
]

# 设置平滑段总长度、轨距、步长等参数
L_smo = 3.0     # 例如3.0 => 左右各1.5m
b_ref = 1.5     # 中心线到钢轨的半轨距
ds    = 0.1     # 积分步长

#===================== 2) 调用生成轨迹 =====================
# generate_trajectory 会返回: (trajectory_data, Kappa, U)
trajectory_data, Kappa, U = generate_trajectory(
    h_segments, 
    u_segments, 
    L_smo=L_smo, 
    b_ref=b_ref, 
    ds=ds
)

#===================== 2.1) 从 trajectory_data 中取出数值并转为 numpy array =====================
s_vals     = np.array(trajectory_data['s'])
xvals      = np.array(trajectory_data['x'])
yvals      = np.array(trajectory_data['y'])
zvals      = np.array(trajectory_data['z'])
psi_vals   = np.array(trajectory_data['psi'])
phi_vals   = np.array(trajectory_data['phi'])
left_rail  = np.array(trajectory_data['left_rail'])
right_rail = np.array(trajectory_data['right_rail'])

#===================== 3) 存储为文件（.npz / .json）可选 =====================
np.savez('trajectory_data.npz', **trajectory_data)

with open('trajectory_data.json', 'w') as jf:
    json.dump(trajectory_data, jf, indent=4)

#===================== 4) 绘制3D轨迹视图 =====================
fig = plt.figure(figsize=(9, 18))

#----- (4.1) 3D视图(不等比例) -----
ax1 = fig.add_subplot(311, projection='3d')
ax1.plot(xvals, yvals, zvals, 'k-', label='CenterLine')
ax1.plot(left_rail[:,0], left_rail[:,1], left_rail[:,2], 'r-', label='Left Rail')
ax1.plot(right_rail[:,0], right_rail[:,1], right_rail[:,2], 'b-', label='Right Rail')
ax1.set_title("3D Track View (Non-Equal Axes)")
ax1.set_xlabel("X [m]")
ax1.set_ylabel("Y [m]")
ax1.set_zlabel("Z [m]")
ax1.legend()

#----- (4.2) 3D视图(等比例) -----
ax2 = fig.add_subplot(312, projection='3d')
ax2.plot(xvals, yvals, zvals, 'k-')
ax2.plot(left_rail[:,0], left_rail[:,1], left_rail[:,2], 'r-')
ax2.plot(right_rail[:,0], right_rail[:,1], right_rail[:,2], 'b-')
ax2.set_title("3D Track View (Equal Axes)")
ax2.set_xlabel("X [m]")
ax2.set_ylabel("Y [m]")
ax2.set_zlabel("Z [m]")

# 强制坐标轴等比例
x_limits = ax2.get_xlim3d()
y_limits = ax2.get_ylim3d()
z_limits = ax2.get_zlim3d()
x_range = abs(x_limits[1] - x_limits[0])
y_range = abs(y_limits[1] - y_limits[0])
z_range = abs(z_limits[1] - z_limits[0])
max_range = max(x_range, y_range, z_range)
mid_x = 0.5*(x_limits[0] + x_limits[1])
mid_y = 0.5*(y_limits[0] + y_limits[1])
mid_z = 0.5*(z_limits[0] + z_limits[1])
ax2.set_xlim(mid_x - max_range/2, mid_x + max_range/2)
ax2.set_ylim(mid_y - max_range/2, mid_y + max_range/2)
ax2.set_zlim(mid_z - max_range/2, mid_z + max_range/2)

#----- (4.3) XY平面俯视图(2D) -----
ax3 = fig.add_subplot(313)
ax3.plot(xvals, yvals, 'k-', label='CenterLine')
ax3.plot(left_rail[:,0], left_rail[:,1], 'r-', label='Left Rail')
ax3.plot(right_rail[:,0], right_rail[:,1], 'b-', label='Right Rail')
ax3.set_aspect('equal', 'box')
ax3.set_title("XY Plane View (Top-Down)")
ax3.set_xlabel("X [m]")
ax3.set_ylabel("Y [m]")
ax3.legend()
ax3.grid(True)

plt.tight_layout(pad=2.0)
plt.show()

# 自主作图与 SIMPACK 原始数据对比
- 需要**先运行**上述作图函数, 以获得 left_rail, right_rail, xvals, yvals 
- SIMPACK GUI 之中，轨道平面图之中，Y在横坐标，X在纵坐标
- **读取** SPCK 导出的水平直线与曲线线路分布的数据文件: TrkHorizontal_R300m60kmph_Vehicle4WDB.txt

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os

from tools_SPCKTrk import read_track_data

# 读取SIMPACK轨道数据
simpack_file = os.path.join(os.getcwd(), 'TrkHorizontal_R300m60kmph_Vehicle4WDB.txt')
splined_track_x, splined_track_y = read_track_data(simpack_file)

# 创建独立的图形来对比数据
plt.figure(figsize=(10, 8))
plt.plot(yvals, xvals, 'b-', linewidth=2, label='Our CenterLine')
plt.plot(splined_track_x, splined_track_y, 'r--', linewidth=2, label='SIMPACK Track')
plt.xlabel("Y [m]")
plt.ylabel("X [m]")
plt.title("Horizontal Track Comparison: Our Implementation vs SIMPACK")
plt.legend()
plt.axis('equal')  # 设置坐标轴等比例
plt.grid(True)     # 增加网格线
plt.tight_layout()
plt.show()

# SIMPACK超高轨道数据可视化
- 红色为 SPCK 导出
- 蓝色为自定义曲线
- 两者有极小误差, 小于 2e-5 m（0.02 mm）
- SPCK 导出的超高随线路延伸的数据文件: TrkSuperlev_R300m60kmph_Vehicle4WDB.txt

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 定义轨道参数
h_s_bounds = [1650]  # 轨道总长，例如1650m
ds = 0.1

# 1. 生成要绘图的 s 序列
s_end = h_s_bounds[-1]
s_vals = np.arange(0, s_end + ds*0.1, ds)

# 2. 对每个 s 计算超高 u(s)
u_vals = [U(s) for s in s_vals]

# 读取SIMPACK超高轨道数据
superlev_file = os.path.join(os.getcwd(), 'TrkSuperlev_R300m60kmph_Vehicle4WDB.txt')
splined_track_x, splined_track_y = read_track_data(superlev_file)

plt.figure(figsize=(10, 6))

# 用蓝色绘制自定义曲线
plt.plot(s_vals, u_vals, 'b-', linewidth=2, label='Our Superelevation u(s)')

# 用红色虚线绘制SIMPACK数据以提高可见性逆向
plt.plot(splined_track_x, splined_track_y, 'r--', linewidth=2, label='SIMPACK Superelevation')

# 保持图例和标签为英文
plt.title("Superelevation Comparison")
plt.xlabel("s [m]")
plt.ylabel("u(s) [m]")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()
